In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [13]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [14]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [15]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [17]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i

In [28]:
def get_accuracy_prob(embed_dim):
    
    pairs = []

    for i in vocabs:
        for j in vocabs:
            for k in vocabs:
                if i != j and i != k and j != k:
                    pairs.append((i,j,k))

    indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

    pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
    pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]
    
    sentences_train = []
    sentences_number_train = []
    sentences_test_a = []
    sentences_number_test_a = []
    sentences_test_b = []
    sentences_number_test_b = []

    x_masked_train = []
    y_masked_labels_train = []
    x_masked_test_a = []
    y_masked_labels_test_a = []
    x_masked_test_b = []
    y_masked_labels_test_b = []

    for _ in range(25000):

        [(a,b,c), (d,e,f)] = random.sample(pairs_train, 2)

        temp = [a, b, c, a, d, e, f, d]
        sentences_train.append(temp)
        sentences_number_train.append([vocab_map[i] for i in temp])
        x_masked_train.append([vocab_map[i] for i in temp])
        y_masked_labels_train.append([vocab_map[i] for i in temp][1:])

        [(a,b,c), (d,e,f)] = random.sample(pairs_train, 2)

        temp = [a, b, c, b, d, e, f, e]
        sentences_train.append(temp)
        sentences_number_train.append([vocab_map[i] for i in temp])
        x_masked_train.append([vocab_map[i] for i in temp])
        y_masked_labels_train.append([vocab_map[i] for i in temp][1:])



    for _ in range(25000):

        [(a,b,c), (d,e,f)] = random.sample(pairs_test, 2)

        temp = [a, b, c, a, d, e, f, d]
        sentences_test_a.append(temp)
        sentences_number_test_a.append([vocab_map[i] for i in temp])
        x_masked_test_a.append([vocab_map[i] for i in temp])
        y_masked_labels_test_a.append([vocab_map[i] for i in temp][1:])

        [(a,b,c), (d,e,f)] = random.sample(pairs_test, 2)

        temp = [a, b, c, b, d, e, f, e]
        sentences_test_b.append(temp)
        sentences_number_test_b.append([vocab_map[i] for i in temp])
        x_masked_test_b.append([vocab_map[i] for i in temp])
        y_masked_labels_test_b.append([vocab_map[i] for i in temp][1:])

    x_masked_train = np.array(x_masked_train)
    y_masked_labels_train = np.array(y_masked_labels_train)
    x_masked_test_a = np.array(x_masked_test_a)
    y_masked_labels_test_a = np.array(y_masked_labels_test_a)
    x_masked_test_b = np.array(x_masked_test_b)
    y_masked_labels_test_b = np.array(y_masked_labels_test_b)

    perm = np.random.permutation(len(x_masked_train))
    x_masked_train = x_masked_train[perm]
    y_masked_labels_train = y_masked_labels_train[perm]
    
    num_head = 2

    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
    sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
    encoder_output = word_embeddings + sinusoidal_embeddings

    for i in range(1):
        encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

    encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
    mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                            validation_split = 0.5, callbacks = [callback], 
                            epochs=2000, batch_size=5000, 
                            verbose=0)
    
    acc_a = []
    prob_a = []
    x_test_subset_a = x_masked_test_a[np.random.choice(x_masked_test_a.shape[0], size=1000, replace=False)]

    for sentence_number in x_test_subset_a:
        temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        temp = temp[:,-1,:]
        acc_a.append(1 if temp.argmax() == sentence_number[-1] else 0)
        prob_a.append(temp[0][sentence_number[-1]])
        
    acc_b = []
    prob_b = []
    x_test_subset_b = x_masked_test_b[np.random.choice(x_masked_test_b.shape[0], size=1000, replace=False)]

    for sentence_number in x_test_subset_b:
        temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        temp = temp[:,-1,:]
        acc_b.append(1 if temp.argmax() == sentence_number[-1] else 0)
        prob_b.append(temp[0][sentence_number[-1]])
        
    return ((np.mean(acc_a), np.mean(prob_a)), (np.mean(acc_b), np.mean(prob_b)))


In [29]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    ((acc_a, prob_a), (acc_b, prob_b)) = get_accuracy_prob(10)
    
    print((acc_a, prob_a))
    print((acc_b, prob_b))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    accs_b += acc_b/10
    probs_b += prob_b/10
    
print((accs_a, probs_a))
print((accs_b, probs_b))

(0.099, 0.16983715)
(0.737, 0.41055194)
(0.113, 0.15490887)
(0.64, 0.34468096)
(0.119, 0.1698606)
(0.704, 0.40527686)
(0.124, 0.17734842)
(0.729, 0.4257678)
(0.137, 0.16303682)
(0.587, 0.3251759)
(0.113, 0.17032285)
(0.678, 0.39980865)
(0.145, 0.15481405)
(0.592, 0.31983876)
(0.086, 0.15395984)
(0.688, 0.37650383)
(0.11, 0.17008868)
(0.72, 0.408492)
(0.144, 0.13132802)
(0.507, 0.24644962)
(0.119, 0.16155052930116656)
(0.6581999999999999, 0.36625463366508476)


In [30]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    ((acc_a, prob_a), (acc_b, prob_b)) = get_accuracy_prob(100)
    
    print((acc_a, prob_a))
    print((acc_b, prob_b))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    accs_b += acc_b/10
    probs_b += prob_b/10
    
print((accs_a, probs_a))
print((accs_b, probs_b))

(0.474, 0.48119578)
(0.504, 0.4957202)
(0.393, 0.46891424)
(0.559, 0.5110974)
(0.52, 0.4802005)
(0.496, 0.47941843)
(0.58, 0.51029897)
(0.413, 0.46803427)
(0.534, 0.50142425)
(0.459, 0.47852844)
(0.559, 0.49172074)
(0.427, 0.45502844)
(0.468, 0.4822211)
(0.496, 0.48650253)
(0.549, 0.50270087)
(0.464, 0.47450814)
(0.528, 0.49245727)
(0.438, 0.4677361)
(0.501, 0.48614782)
(0.506, 0.5010683)
(0.5106, 0.4897281527519226)
(0.47619999999999996, 0.4817642211914062)
